# Project No. 3 - Bouquet of Flowers
### Authors:
M. Alejandro Villalobos C.
Óscar Ruiz Ramirez
Sofía Vargas Aceves
### Fecha:
24 de Marzo, 2022
### Description:
Tercer proyecto Machine Learning. Bouquet of Flowers Classification

## 1. DATA READ


In [2]:
import pandas as pd

Dataset = pd.read_csv("./Embedded_images.csv")


Dataset.keys()

Index(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9',
       ...
       'n2044', 'n2045', 'n2046', 'n2047', 'category', 'image name', 'image',
       'size', 'width', 'height'],
      dtype='object', length=2054)

In [3]:
Dataset = Dataset.drop(['image','image name', 'size', 'width','height'],axis=1)
Dataset['category'] = Dataset['category'].replace(['Camelia', 'Dahlia', 'Hydrangea', 'Lilies', 'Orchids', 'Peony', 'Ranunculus', 'Roses', 'Sunflowers', 'Tulips'], [0,1,2,3,4,5,6,7,8,9])
DataFrameFlowers = pd.DataFrame(Dataset)


## 2. Data Preprocessing

In [4]:
X = DataFrameFlowers.drop(['category'], axis = 1)
Y = DataFrameFlowers['category']


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=0)

## Model #1 Logistic Regression

### 3. Model Creation

In [5]:
# Implementado con SAGA por el tamaño de muestra, referenciando (6)
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=50.0 / y_train.shape[0], penalty="l1", solver="saga", tol=0.1)
LR_better = LogisticRegression(C=50.0 / y_train.shape[0], penalty="l2", solver="lbfgs", tol=1e-4)


### 4. Training: Adjust Model with Historic Data

In [6]:
LR.fit(X_train, y_train)
LR_better.fit(X_train, y_train)

D:\Programs\Code\anaconda3\envs\DeepEnv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.24875621890547264)

### 5. Prediction for New Data

In [7]:
y_predictLR = LR.predict(X_test)
y_predictLR_better = LR_better.predict(X_test)

## Model #2 Convolutional Neural Network (CNN)

In [8]:
#Aqui haremos las variables especialmente para el modelo CNN, ya que se necesita estar en 3 dimensiones
#Para training 201 x 2048 x 1, para test 51 x 2048 x 1
#Código complementado con (5)
X_trainCNN = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_testCNN = X_test.values.reshape(X_test.shape[0], X_test.shape[1] ,1)

In [9]:
X_train.shape

(201, 2048)

### 3. Model Creation

In [108]:
import numpy as np
import keras 
from keras.models import Sequential
from keras.layers import  Dense, Conv1D, Flatten, MaxPooling1D, Dropout

#Para coincidir tamaños, se convierten las etiquetas en "one hot vectors"
#Código complementado con (5)
y_trainCNN = keras.utils.to_categorical(np.asarray(y_train.factorize()[0]))
y_testCNN = keras.utils.to_categorical(np.asarray(y_test.factorize()[0]))

CNN = Sequential()
CNN.add(Conv1D(128, 3, activation='relu', input_shape=(2048,1)))
CNN.add(MaxPooling1D((2)))
CNN.add(Conv1D(256, 3, activation='relu'))
CNN.add(MaxPooling1D((2)))
CNN.add(Conv1D(256, 3, activation='relu'))
CNN.add(MaxPooling1D((2)))

CNN.add(Flatten())
CNN.add(Dense(256, activation='relu'))

#Capa Dense tamaño 10 por las 10 categorizaciones finales
#Complementado con activación softmax según (7)
CNN.add(Dense(10, activation='softmax'))

#Complementado con loss según (7)
CNN.compile(optimizer='adam', loss = keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [109]:
n_timesteps, n_features, n_outputs = X_trainCNN.shape[1], X_trainCNN.shape[2], y_trainCNN.shape[1]
CNN_better = Sequential()
CNN_better.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
CNN_better.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
CNN_better.add(Dropout(0.7))
CNN_better.add(MaxPooling1D(pool_size=2))
CNN_better.add(Flatten())
CNN_better.add(Dense(256, activation='relu'))

#Capa Dense tamaño 10 por las 10 categorizaciones finales
#Complementado con activación softmax según (7)
CNN_better.add(Dense(n_outputs, activation='softmax'))

#Complementado con loss según (7)
CNN_better.compile(optimizer='adam', loss = keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [110]:
y_trainCNN.shape

(201, 10)

### 4. Training: Adjust Model with Historic Data

In [111]:
CNN_H = CNN.fit(X_trainCNN, y_trainCNN, validation_data=(X_testCNN, y_testCNN), epochs=10)

Epoch 1/10
7/7 [==============================] - 5s 746ms/step - loss: 2.5153 - accuracy: 0.1493 - val_loss: 2.2982 - val_accuracy: 0.1569
Epoch 2/10
7/7 [==============================] - 4s 600ms/step - loss: 2.1568 - accuracy: 0.3433 - val_loss: 2.1997 - val_accuracy: 0.3725
Epoch 3/10
7/7 [==============================] - 5s 667ms/step - loss: 1.6505 - accuracy: 0.4975 - val_loss: 2.1409 - val_accuracy: 0.2941
Epoch 4/10
7/7 [==============================] - 5s 726ms/step - loss: 0.9918 - accuracy: 0.6716 - val_loss: 2.8746 - val_accuracy: 0.2745
Epoch 5/10
7/7 [==============================] - 6s 837ms/step - loss: 0.5380 - accuracy: 0.8607 - val_loss: 3.9956 - val_accuracy: 0.3333
Epoch 6/10
7/7 [==============================] - 5s 729ms/step - loss: 0.3674 - accuracy: 0.8657 - val_loss: 3.6519 - val_accuracy: 0.3725
Epoch 7/10
7/7 [==============================] - 4s 612ms/step - loss: 0.1723 - accuracy: 0.9602 - val_loss: 4.4047 - val_accuracy: 0.3529
Epoch 8/10
7/7 [====

In [112]:
CNN_HBetter = CNN_better.fit(X_trainCNN, y_trainCNN, validation_data=(X_testCNN, y_testCNN), epochs=10)

Epoch 1/10
7/7 [==============================] - 2s 343ms/step - loss: 2.7477 - accuracy: 0.2388 - val_loss: 2.5939 - val_accuracy: 0.1569
Epoch 2/10
7/7 [==============================] - 2s 305ms/step - loss: 1.8660 - accuracy: 0.5323 - val_loss: 2.1788 - val_accuracy: 0.1961
Epoch 3/10
7/7 [==============================] - 3s 362ms/step - loss: 1.1228 - accuracy: 0.7363 - val_loss: 2.4393 - val_accuracy: 0.2353
Epoch 4/10
7/7 [==============================] - 2s 318ms/step - loss: 0.5527 - accuracy: 0.8408 - val_loss: 2.6968 - val_accuracy: 0.3922
Epoch 5/10
7/7 [==============================] - 2s 351ms/step - loss: 0.3094 - accuracy: 0.8806 - val_loss: 3.4925 - val_accuracy: 0.3137
Epoch 6/10
7/7 [==============================] - 2s 287ms/step - loss: 0.1940 - accuracy: 0.9353 - val_loss: 3.6996 - val_accuracy: 0.3137
Epoch 7/10
7/7 [==============================] - 2s 299ms/step - loss: 0.1257 - accuracy: 0.9552 - val_loss: 3.7473 - val_accuracy: 0.3333
Epoch 8/10
7/7 [====

### 5. Prediction for New Data

In [113]:
y_predictCNN = CNN.predict_classes(X_testCNN)
y_predictCNN_better = CNN_better.predict_classes(X_testCNN)

## Model #3 Support Vector Machine (SVM)

### 3. Model Creation

In [114]:
from sklearn import svm
SVM = svm.SVC(kernel='poly')
SVM_better = svm.SVC(kernel='linear')

### 4. Training: Adjust Model with Historic Data

In [115]:
SVM.fit(X_train, y_train)
SVM_better.fit(X_train, y_train)

SVC(kernel='linear')

### 5. Prediction for New Data

In [116]:
y_predictSVM = SVM.predict(X_test)
y_predictSVM_better = SVM_better.predict(X_test)

## Model #4 Random Forest

### 3. Model Creation

In [117]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=500)
RFC_better = RandomForestClassifier(n_estimators= 400,max_features= 'sqrt',bootstrap= False)

### 4. Training: Adjust Model with Historic Data

In [118]:
RFC.fit(X_train, y_train)
RFC_better.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_features='sqrt', n_estimators=400)

### 5. Prediction for New Data

In [119]:
y_predictRF = RFC.predict(X_test)
y_predictRF_better = RFC_better.predict(X_test)

## 6. Visualization of Results

In [120]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

### Model #1 Logistic Regression

In [121]:
CV_LR = cross_val_score(LR, X_train, y_train, cv=3, scoring = "accuracy")
CV_LR_better = cross_val_score(LR_better, X_train, y_train, cv=3, scoring = "accuracy")

D:\Programs\Code\anaconda3\envs\DeepEnv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Programs\Code\anaconda3\envs\DeepEnv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

In [122]:
AS_LR = accuracy_score(y_test,y_predictLR)
AS_LR_better = accuracy_score(y_test,y_predictLR_better)

### Model #2 Support Vector Machine (SVM)

In [123]:
CV_SVM = cross_val_score(SVM, X_train, y_train, cv=3, scoring = "accuracy")
CV_SVM_better = cross_val_score(SVM_better, X_train, y_train, cv=3, scoring = "accuracy")

In [124]:
AS_SVM = accuracy_score(y_test,y_predictSVM)
AS_SVM_better = accuracy_score(y_test,y_predictSVM_better)

### Model #3 Random Forest

In [125]:
CV_RF = cross_val_score(RFC, X_train, y_train, cv=3, scoring = "accuracy")
CV_RF_better = cross_val_score(RFC_better, X_train, y_train, cv=3, scoring = "accuracy")

In [126]:
AS_RF = accuracy_score(y_test,y_predictRF)
AS_RF_better = accuracy_score(y_test,y_predictRF_better)

### Model #5 Convolutional Neural Network (CNN)

In [127]:
CV_CNN = CNN.evaluate(X_trainCNN, y_trainCNN)
CV_CNN_better = CNN_better.evaluate(X_trainCNN, y_trainCNN)

7/7 [==============================] - 0s 48ms/step - loss: 0.0520 - accuracy: 0.9851


In [128]:
AS_CNN = CNN.evaluate(X_testCNN, y_testCNN)
AS_CNN_better = CNN_better.evaluate(X_testCNN, y_testCNN)

2/2 [==============================] - 0s 17ms/step - loss: 3.8863 - accuracy: 0.3333


In [129]:
#Creación de dataframe con los resultados
models = ["LR","SVM", "RF", "CNN","LR_better", "SVM_better", "RF_better", "CNN_better"]
AS_values = [AS_LR, AS_SVM, AS_RF,  AS_CNN[1], AS_LR_better, AS_SVM_better, AS_RF_better, AS_CNN_better[1]]
CV_values = [CV_LR, CV_SVM, CV_RF, [CV_CNN[1],'-','-'],CV_LR_better,CV_SVM_better, CV_RF_better, [CV_CNN_better[1],'-','-']]

data = {"Accuracy Scores": AS_values, "Cross Validations": CV_values}
df = pd.DataFrame(data, index = models)

#Separación de arrays de Cross Validations en columnas, referenciando (10)
dfCV = pd.DataFrame(df['Cross Validations'].to_list(), columns=['CV #1','CV #2','CV #3'], index = models)
#Cálculo de promedio de Cross Validations
dfCV.insert(0,'CV AVG', dfCV[['CV #1','CV #2','CV #3']].mean(axis=1, numeric_only=True))

#Concatenación de dataframes con las columnas finales, referenciando (11)
dfAcc = pd.concat([df['Accuracy Scores'], dfCV], axis=1)
dfAcc = dfAcc.style.set_caption("Models' Accuracy Scores and Cross Validations")
dfAcc

,Accuracy Scores,CV AVG,CV #1,CV #2,CV #3
LR,0.666667,0.582090,0.582090,0.656716,0.731343
SVM,0.588235,0.716418,0.716418,0.716418,0.746269
RF,0.745098,0.671642,0.671642,0.701493,0.686567
CNN,0.352941,0.985075,0.985075,-,-
LR_better,0.745098,0.716418,0.716418,0.731343,0.761194
SVM_better,0.686275,0.746269,0.746269,0.746269,0.731343
RF_better,0.647059,0.641791,0.641791,0.701493,0.701493
CNN_better,0.333333,0.985075,0.985075,-,-


## Confusion Matrix


In [130]:
from sklearn.metrics import confusion_matrix
cmLR=confusion_matrix(y_test,y_predictLR)
print(cmLR)

print("--------------------------------")
cmLR_better=confusion_matrix(y_test,y_predictLR_better)
print(cmLR_better)

[[3 0 0 1 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [1 0 4 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 2 1 2 0 0 0 0 0]
 [2 0 0 0 0 2 0 0 1 2]
 [0 0 0 2 0 1 3 1 0 0]
 [0 0 0 0 0 0 1 8 0 0]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 2 0 4]]
--------------------------------
[[3 0 0 0 1 0 0 0 0 0]
 [0 2 0 0 1 0 0 0 0 0]
 [0 0 5 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 0 1 4 0 0 0 0 0]
 [1 0 0 0 0 5 0 0 0 1]
 [0 0 0 1 1 1 4 0 0 0]
 [0 0 0 1 0 0 2 5 0 1]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 1 0 0 5]]


Se puede apreciar en esta matriz de confusion que al sumar la cantidad de positivos, osea de los valores correctos que mostro (La diagonal) nos dio mayor cantidad el segundo modelo de logistic Regression. 

In [134]:
cmCNN=confusion_matrix(y_test,y_predictCNN)
print(cmCNN)

print("--------------------------------")
cmCNN_better=confusion_matrix(y_test,y_predictCNN_better)
print(cmCNN_better)

[[0 0 0 0 0 4 0 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 4 0 0 1 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 3 0 0 0 0 0 2 0]
 [0 0 0 0 1 1 0 1 0 4]
 [3 1 0 1 0 0 0 0 1 1]
 [3 1 0 5 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0]
 [0 0 0 3 2 0 0 0 1 0]]
--------------------------------
[[0 0 0 0 0 4 0 0 0 0]
 [0 0 0 0 0 0 0 2 1 0]
 [0 0 4 0 0 1 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 3 0]
 [0 0 0 0 2 1 0 0 0 4]
 [3 1 0 0 0 0 0 0 1 2]
 [3 1 0 4 1 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0]
 [1 0 0 0 4 0 0 0 1 0]]


Aha

In [131]:
cmSVM=confusion_matrix(y_test,y_predictSVM)
print(cmSVM)

print("--------------------------------")
cmSVM_better=confusion_matrix(y_test,y_predictSVM_better)
print(cmSVM_better)

[[3 0 0 1 0 0 0 0 0 0]
 [0 2 0 0 1 0 0 0 0 0]
 [1 0 4 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 0 1 4 0 0 0 0 0]
 [1 0 0 0 0 3 0 0 1 2]
 [0 0 0 2 0 1 3 1 0 0]
 [0 0 0 3 0 0 3 3 0 0]
 [0 1 0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 1 0 4]]
--------------------------------
[[3 0 0 1 0 0 0 0 0 0]
 [0 2 0 0 1 0 0 0 0 0]
 [1 0 4 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 0 1 4 0 0 0 0 0]
 [1 0 0 0 0 5 0 0 0 1]
 [0 0 0 1 1 1 4 0 0 0]
 [0 0 0 3 0 0 3 3 0 0]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 1 0 5]]


Se puede apreciar en esta matriz de confusion que al sumar la cantidad de positivos, osea de los valores correctos que mostro (La diagonal) nos dio mayor cantidad el segundo modelo de Support Vector Machine. 

In [133]:
cmRF=confusion_matrix(y_test,y_predictRF)
print(cmRF)

print("--------------------------------")
cmRF_better=confusion_matrix(y_test,y_predictRF_better)
print(cmRF_better)

[[4 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 5 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 0 1 4 0 0 0 0 0]
 [1 0 0 0 0 3 0 0 1 2]
 [0 0 0 1 0 1 5 0 0 0]
 [0 0 0 1 0 0 2 5 0 1]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 1 0 0 0 1 0 4]]
--------------------------------
[[3 0 0 1 0 0 0 0 0 0]
 [0 2 0 0 1 0 0 0 0 0]
 [0 0 4 0 0 1 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 0 1 3 0 0 0 0 1]
 [1 0 0 0 0 3 0 0 1 2]
 [0 0 0 1 0 2 3 1 0 0]
 [0 0 0 2 0 0 1 6 0 0]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 2 0 0 0 0 0 4]]


a